# Applied Data Scince Capstone Week 3 Lab

## Part 1

### Install beautifulsoup and other required libs for web scraping

In [10]:
!pip install lxml html5lib beautifulsoup4

### Install Pandas and read data frames from wikiedia page

In [11]:
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
print(len(dfs))

3


### Select the appropraite data frame

In [15]:
print(dfs[0])

    Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
..          ...               ...   
175         M5Z      Not assigned   
176         M6Z      Not assigned   
177         M7Z      Not assigned   
178         M8Z         Etobicoke   
179         M9Z      Not assigned   

                                         Neighbourhood  
0                                         Not assigned  
1                                         Not assigned  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
..                                                 ...  
175                                       Not assigned  
176                                       Not assigned  
177                                       

### Assign the data frame and filter out rows where Borough is not assigned

In [71]:
df1 = dfs[0]
mask = df1['Borough'] != 'Not assigned'
df2 = df1[mask]
df2

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Group by postal code



In [72]:
df3 = df2.groupby("Postal Code").sum()
df3

,Borough,Neighbourhood
Postal Code,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
...,...,...
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."


### Reset index



In [66]:
df4 = df3.reset_index()
df4

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


### Print the shape of the dataframe

In [123]:
df4.shape

(103, 5)

# ========================================

## Part 2

### Get the geolocations

In [124]:
import requests
from pandas.io.json import json_normalize

In [128]:
url = "http://geogratis.gc.ca/services/geolocation/en/locate?q=M1B"
resp = requests.get(url=url)

# Handling nested JSON:
df = pd.json_normalize(resp.json())
df.head()

,title,qualifier,type,geometry.type,geometry.coordinates
0,M1B,INTERPOLATED_CENTROID,ca.gc.nrcan.geoloc.data.model.PostalCode,Point,"[-79.193321, 43.809444]"


In [169]:
url = "http://geogratis.gc.ca/services/geolocation/en/locate?q=M1B"
resp = requests.get(url=url)

# Handling nested JSON:
df = pd.json_normalize(resp.json())
#df.head()
print(df['geometry.coordinates'][0][1])
print(df['geometry.coordinates'][0][0])

43.809444
-79.193321


In [138]:
url = "http://geogratis.gc.ca/services/geolocation/en/locate?q="
print(url + df4["Postal Code"][1])

http://geogratis.gc.ca/services/geolocation/en/locate?q=M1C


In [146]:
for postal_code in df4["Postal Code"]:
    print(postal_code)

M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X
M2H
M2J
M2K
M2L
M2M
M2N
M2P
M2R
M3A
M3B
M3C
M3H
M3J
M3K
M3L
M3M
M3N
M4A
M4B
M4C
M4E
M4G
M4H
M4J
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5M
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6A
M6B
M6C
M6E
M6G
M6H
M6J
M6K
M6L
M6M
M6N
M6P
M6R
M6S
M7A
M7R
M7Y
M8V
M8W
M8X
M8Y
M8Z
M9A
M9B
M9C
M9L
M9M
M9N
M9P
M9R
M9V
M9W


M7R returns an empty JSON file
Latitude, Longitude coordinates are:
M7R = 43.63705212384915, -79.61562720000305

In [175]:
lat = []
long = []
url_base = "http://geogratis.gc.ca/services/geolocation/en/locate?q="

for postal_code in df4["Postal Code"]:
    if postal_code == 'M7R':
        lat.append('43.63705212384915')
        long.append('-79.61562720000305')
    else:
        url = url_base + postal_code
        #print(url)

        resp = requests.get(url)
        df = pd.json_normalize(resp.json())
    
        lat.append(df['geometry.coordinates'][0][1])
        long.append(df['geometry.coordinates'][0][0])
    
    #latitude = df['geometry.coordinates'][0][1]
    #longitude = df['geometry.coordinates'][0][0]
    
    #lat.append(latitude)
    #long.append(longitude)
#print(lat)
#print(long)

df4['Latitude'] = lat
df4['Longitude'] = long
df4

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8094,-79.1933
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.788,-79.1587
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7697,-79.1731
3,M1G,Scarborough,Woburn,43.7724,-79.2151
4,M1H,Scarborough,Cedarbrae,43.7713,-79.2408
...,...,...,...,...,...
98,M9N,York,Weston,43.7067,-79.52
99,M9P,Etobicoke,Westmount,43.6984,-79.5343
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.6897,-79.5572
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.7454,-79.5865


# ========================================

## Part 3